In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sts

In [2]:
x_distr = sts.rv_discrete(values=[
    (1, 3),
    (0.2, 0.8)
])
x_distr

In [3]:
n_distr = sts.poisson(1)
n_distr.mean()

1.0

# Основной метод

In [4]:
def convolve(f, g):
    a = f.a + g.a
    b = f.b + g.b
    values = list(range(a, b + 1))
    probas = []
    for x in values:
        p = sum([f.pmf(x - k) * g.pmf(k) for k in range(x)])
        probas.append(p)
    
    return sts.rv_discrete(values=(values, probas))

In [5]:
convs = [
    sts.rv_discrete(values=([0], [1])),
    ]
for i in range(6):
    convs.append(convolve(x_distr, convs[-1]))

In [6]:
tab = pd.concat([pd.Series(data=c.pk, index=c.xk) for c in convs], axis=1).fillna(0)
tab

,0,1,2,3,4,5,6
0,1.0,0.0,0.00,0.000,0.0000,0.00000,0.000000
1,0.0,0.2,0.00,0.000,0.0000,0.00000,0.000000
2,0.0,0.0,0.04,0.000,0.0000,0.00000,0.000000
3,0.0,0.8,0.00,0.008,0.0000,0.00000,0.000000
4,0.0,0.0,0.32,0.000,0.0016,0.00000,0.000000
5,0.0,0.0,0.00,0.096,0.0000,0.00032,0.000000
6,0.0,0.0,0.64,0.000,0.0256,0.00000,0.000064
7,0.0,0.0,0.00,0.384,0.0000,0.00640,0.000000
8,0.0,0.0,0.00,0.000,0.1536,0.00000,0.001536
9,0.0,0.0,0.00,0.512,0.0000,0.05120,0.000000


In [7]:
tab.sum(axis=0)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
dtype: float64

In [8]:
pd.Series((tab.values * n_distr.pmf(tab.columns).reshape((1, -1))).sum(axis=1))

0     0.367879
1     0.073576
2     0.007358
3     0.294794
4     0.058885
5     0.005887
6     0.118114
7     0.023564
8     0.002355
9     0.031549
10    0.006286
11    0.000628
12    0.006320
13    0.001256
14    0.000126
15    0.001005
16    0.000201
17    0.000000
18    0.000134
dtype: float64